### Building the Graph

1）variable： 用于存储两个变量
   训练次数 （不参与训练）
   output
   
2）transformation： 对输入参数进行变形
   采用 placeholder

3）update： 更新的规律

4）summary 用于 tensor board scalars

5）global_operations 
   初始化参数
   merge summary

In [14]:
import tensorflow as tf
import numpy as np

In [15]:
#创建一个新的Graph/非必须，可用默认：
jimmy_graph=tf.Graph()

In [16]:
with jimmy_graph.as_default():
    #使用name_scope，实现层次化管理
    with tf.name_scope('variables'):
        #变量表示graph运行的次数
        global_step=tf.Variable(0,dtype=tf.int32,trainable=False,name='global_step')
        #变量表示输出的总和
        total_outputs=tf.Variable(0,dtype=tf.float32,trainable=False,name='total_outputs')
    
    # This name scope contains varibales for transformation
    with tf.name_scope('transformation'):
        #input name scope
        with tf.name_scope('input'):
            a=tf.placeholder(tf.float32,shape=(None),name='input_placeholder_a')
        #middle name scope
        with tf.name_scope('middle'):
            b=tf.reduce_prod(a,name='production_b')
            c=tf.reduce_sum(a,name='sum_c')
        #output name scope
        with tf.name_scope('output'):
            output=tf.add(b,c,name='output')
    #This name scope used to update variables    
    with tf.name_scope('update'):
        #add output to total_outputs
        update_total=total_outputs.assign_add(output)
        total_step=global_step.assign_add(1)
        
    #This name scope create tensorboard summaries
    with tf.name_scope('summaries'):
        average=tf.div(update_total,tf.cast(total_step,tf.float32),name='average')
        tf.summary.scalar('output_summary',output)
        tf.summary.scalar ('total_summary',update_total)
        tf.summary.scalar ('average_summary',average)
    
    with tf.name_scope('global_opeartion'):
        #初始化
        init=tf.global_variables_initializer()
        #合并所有summary
        merged_summaries=tf.summary.merge_all()    

In [17]:
sess=tf.Session(graph=jimmy_graph)

In [18]:
summary_writer=tf.summary.FileWriter('./jimmy_graph',sess.graph)
sess.run(init)

In [31]:
def run_graph(input_tensor):
    feed_dict={a:input_tensor}
    _,step,summary=sess.run([output,total_step,merged_summaries],feed_dict=feed_dict)
    summary_writer.add_summary(summary,global_step=step)
    summary_writer.add_graph(sess.graph)

In [32]:
run_graph([2,8])

In [33]:
# Call this method to make sure that all pending events have been written to disk.
summary_writer.flush()

In [34]:
summary_writer.close()
sess.close()

In [35]:
print (tf.get_default_graph().as_graph_def())

versions {
  producer: 26
}



![avait](https://imgsa.baidu.com/forum/w%3D580/sign=a23356fd3a7adab43dd01b4bbbd5b36b/d216b926cffc1e17d0a869e44690f603728de914.jpg)